In [1]:
from truss_torch import *

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

joints = [
    Joint(JointType.PIN),  # 0
    Joint(JointType.GUSSET), # 1
    Joint(JointType.GUSSET), # 2
    Joint(JointType.GUSSET), # 3
    Joint(JointType.ROLLER), # 4 
    Joint(JointType.GUSSET), # 5
    Joint(JointType.GUSSET), # 6
    Joint(JointType.GUSSET), # 7
    Joint(JointType.GUSSET), # 8
    Joint(JointType.GUSSET)] # 9


links = [Link(0, 1), Link(1,2), Link(2,3), Link(3,4), Link(0,5), Link(1,5), Link(1,6), Link(2,6), Link(2,7), Link(2,8), Link(3,8), Link(3,9), Link(4,9), Link(5,6), Link(6,7), Link(7,8), Link(8,9)]

FORCE_PER_M = 2.5
bridge_joints = [0, 1, 2, 3, 4]
train_force = Force(fx=0, fy=FORCE_PER_M, joints=bridge_joints)

forces = [train_force]

truss = Truss(joints, links, forces, DEVICE)

In [2]:
ground_truth_with_knowns = torch.tensor([[0, 3.5, 7, 10.5, 14, -6.6+7, -4.1+7, 7, 4.9+7, 6.4+7, 0, 0, 0, 0, 0, -1.9, -4, -4.9, -4, -1.9]], requires_grad=True).to(DEVICE)
batched_ground_truth_with_knowns = torch.tensor([[0, 3.5, 7, 10.5, 14, -6.6+7, -4.1+7, 7, 4.9+7, 6.4+7, 0, 0, 0, 0, 0, -1.9, -4, -4.9, -4, -1.9],[0, 3.5, 7, 10.5, 14, -6.6+7, -4.1+7, 7, 4.9+7, 6.4+7, 0, 0, 0, 0, 0, -1.9, -4, -4.9, -4, -1.9]], requires_grad=True).to(DEVICE)
ground_truth = torch.tensor([[3.5, 7, 10.5, -6.6+7, -4.1+7, 7, 4.9+7, 6.4+7, -1.9, -4, -4.9, -4, -1.9]]).to(DEVICE)
known_indices = [1, 2, 3, 5, 6, 7, 8, 9, 15, 16, 17, 18, 19]
replace_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
n_unknowns = 13

def remove_unknowns(X):
    return X[:, known_indices]

def add_knowns(X):
    Y = ground_truth_with_knowns.repeat(torch.Size([X.size(dim=0), 1]))
    Y[:, known_indices] = X[:, replace_indices]
    return Y

In [7]:
class OptimizerNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        layers = [
            nn.Linear(input_dim, hidden_dim), 
            nn.BatchNorm1d(hidden_dim), 
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(hidden_dim, hidden_dim), 
            nn.BatchNorm1d(hidden_dim), 
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(hidden_dim, output_dim)
        ]
        
        for layer in layers:
            if type(layer) == nn.Linear:
                nn.init.kaiming_normal_(layer.weight)

        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

In [14]:

net = OptimizerNet(batched_ground_truth_with_knowns.size(dim=1), 128, batched_ground_truth_with_knowns.size(dim=1))
net.to(DEVICE)
opti = optim.Adam(net.parameters(), lr=0.001)
epochs = 1000
for i in range(epochs):
    net.train()    
    opti.zero_grad()
    X = net(batched_ground_truth_with_knowns)
    print(X)
    loss = truss.solve(X)
    loss.mean().backward()
    opti.step()
    print("Epoch: {0}/{1} loss={2}".format(i, epochs, loss.mean()))

tensor([[-0.0652,  0.0617,  0.0800,  0.0063, -0.0270,  0.0660, -0.0072, -0.0296,
         -0.0294,  0.0140, -0.0244, -0.0015,  0.0323,  0.0137, -0.0498, -0.0656,
         -0.0521, -0.0416,  0.0215, -0.0770],
        [-0.0652,  0.0617,  0.0800,  0.0063, -0.0270,  0.0660, -0.0072, -0.0296,
         -0.0294,  0.0140, -0.0244, -0.0015,  0.0323,  0.0137, -0.0498, -0.0656,
         -0.0521, -0.0416,  0.0215, -0.0770]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
Epoch: 0/1000 loss=71.18761444091797
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
Epoch: 1/1000 loss=nan
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan

KeyboardInterrupt: 